<a href="https://colab.research.google.com/github/ozj1/Machine_Learning_and_Data_Science_projects/blob/main/Chat-Bot_natural_language_processing/Chat_Bot_natural_language_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Loading the Data

We will be working with the Babi Data Set from Facebook Research.

Full Details: https://research.fb.com/downloads/babi/

- Jason Weston, Antoine Bordes, Sumit Chopra, Tomas Mikolov, Alexander M. Rush,
  "Towards AI-Complete Question Answering: A Set of Prerequisite Toy Tasks",
  http://arxiv.org/abs/1502.05698


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
%pwd

'/content'

In [9]:
import pickle
import numpy as np

In [15]:
with open("drive/MyDrive/Dataset/chatbot_dataset/train_qa.txt", "rb") as fp:   # Unpickling
    train_data =  pickle.load(fp)

In [16]:
with open("drive/MyDrive/Dataset/chatbot_dataset/test_qa.txt", "rb") as fp:   # Unpickling
    test_data =  pickle.load(fp)

## understanding the dataset format, a short story, a question, and an answer Yes or No

In [18]:
print(type(test_data))
print(type(train_data))
print(len(test_data))
print(len(train_data))

<class 'list'>
<class 'list'>
1000
10000


In [19]:
train_data[0]

(['Mary',
  'moved',
  'to',
  'the',
  'bathroom',
  '.',
  'Sandra',
  'journeyed',
  'to',
  'the',
  'bedroom',
  '.'],
 ['Is', 'Sandra', 'in', 'the', 'hallway', '?'],
 'no')

In [24]:
# reshaping to get the sentence
print(' '.join(train_data[0][0]))
print(' '.join(train_data[0][1]))
#now what is the answer?
print(train_data[0][2])

Mary moved to the bathroom . Sandra journeyed to the bedroom .
Is Sandra in the hallway ?
no


## Let's see how many unique words we have in the entire dataset using set() and unite them using union().

In [25]:
# Create a set that holds the vocab words
vocab = set()
all_data = test_data + train_data

for story, question , answer in all_data:
    # In case you don't know what a union of sets is:
    # https://www.programiz.com/python-programming/methods/set/union
    vocab = vocab.union(set(story))
    vocab = vocab.union(set(question))

#answer dataset only has two words, we add them
vocab.add('no')
vocab.add('yes')

vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [27]:
vocab_len = len(vocab) + 1 #we add an extra space to hold a 0 for Keras's pad_sequences
#we find the max size of a story
max_story_len = max([len(data[0]) for data in all_data])
print(max_story_len)
#we find the max size of a question in the dataset
max_question_len = max([len(data[1]) for data in all_data])
print(max_question_len)

156
6


## We vectorize the data so it can be understood by Keras.

In [32]:
# Reserve 0 for pad_sequences
vocab_size = len(vocab) + 1

from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

# integer encode sequences of words
tokenizer = Tokenizer(filters=[])
tokenizer.fit_on_texts(vocab)

tokenizer.word_index

{'milk': 1,
 'john': 2,
 'no': 3,
 'is': 4,
 '?': 5,
 'kitchen': 6,
 'grabbed': 7,
 'back': 8,
 'daniel': 9,
 'took': 10,
 'picked': 11,
 'to': 12,
 'bathroom': 13,
 '.': 14,
 'travelled': 15,
 'sandra': 16,
 'in': 17,
 'went': 18,
 'put': 19,
 'moved': 20,
 'yes': 21,
 'garden': 22,
 'dropped': 23,
 'office': 24,
 'mary': 25,
 'football': 26,
 'apple': 27,
 'journeyed': 28,
 'down': 29,
 'there': 30,
 'the': 31,
 'bedroom': 32,
 'discarded': 33,
 'hallway': 34,
 'got': 35,
 'left': 36,
 'up': 37}

In [35]:
train_story_text = []
train_question_text = []
train_answers = []
# We make separate lists for story, query, and answer.
for story,question,answer in train_data:
    train_story_text.append(story)
    train_question_text.append(question)

In [36]:
train_story_seq = tokenizer.texts_to_sequences(train_story_text)

print(len(train_story_text))
print(len(train_story_seq))

10000
10000


## Functionalize Vectorization

In [37]:
def vectorize_stories(data, word_index=tokenizer.word_index, max_story_len=max_story_len,max_question_len=max_question_len):
    '''
    INPUT: 
    
    data: consisting of Stories,Queries,and Answers
    word_index: word index dictionary from tokenizer
    max_story_len: the length of the longest story (used for pad_sequences function)
    max_question_len: length of the longest question (used for pad_sequences function)


    OUTPUT:
    
    Vectorizes the stories,questions, and answers into padded sequences. We first loop for every story, query , and
    answer in the data. Then we convert the raw words to an word index value. Then we append each set to their appropriate
    output list. Then once we have converted the words to numbers, we pad the sequences so they are all of equal length.
    
    Returns this in the form of a tuple (X,Xq,Y) (padded based on max lengths)
    '''
    
    
    # X = STORIES
    X = []
    # Xq = QUERY/QUESTION
    Xq = []
    # Y = CORRECT ANSWER
    Y = []
    
    
    for story, query, answer in data:
        
        # Grab the word index for every word in story
        x = [word_index[word.lower()] for word in story]
        # Grab the word index for every word in query
        xq = [word_index[word.lower()] for word in query]
        
        # Grab the Answers (either Yes/No so we don't need to use list comprehension here)
        # Index 0 is reserved so we're going to use + 1
        y = np.zeros(len(word_index) + 1)
        
        # Now that y is all zeros and we know its just Yes/No , we can use numpy logic to create this assignment
        #
        y[word_index[answer]] = 1
        
        # Append each set of story,query, and answer to their respective holding lists
        X.append(x)
        Xq.append(xq)
        Y.append(y)
        
    # Finally, pad the sequences based on their max length so the RNN can be trained on uniformly long sequences.
        
    # RETURN TUPLE FOR UNPACKING
    return (pad_sequences(X, maxlen=max_story_len),pad_sequences(Xq, maxlen=max_question_len), np.array(Y))

In [38]:
inputs_train, queries_train, answers_train = vectorize_stories(train_data)

In [39]:
inputs_test, queries_test, answers_test = vectorize_stories(test_data)

In [40]:
inputs_test

array([[ 0,  0,  0, ..., 31, 32, 14],
       [ 0,  0,  0, ..., 31, 22, 14],
       [ 0,  0,  0, ..., 31, 22, 14],
       ...,
       [ 0,  0,  0, ..., 31, 27, 14],
       [ 0,  0,  0, ..., 31, 22, 14],
       [ 0,  0,  0, ..., 27, 30, 14]], dtype=int32)

In [41]:
queries_test

array([[ 4,  2, 17, 31,  6,  5],
       [ 4,  2, 17, 31,  6,  5],
       [ 4,  2, 17, 31, 22,  5],
       ...,
       [ 4, 25, 17, 31, 32,  5],
       [ 4, 16, 17, 31, 22,  5],
       [ 4, 25, 17, 31, 22,  5]], dtype=int32)

In [42]:
answers_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [43]:
sum(answers_test)

array([  0.,   0.,   0., 503.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 497.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.])

In [44]:
tokenizer.word_index['yes']

21

In [45]:
tokenizer.word_index['no']

3